<a href="https://colab.research.google.com/github/dfan14051/SUPA-HOT-GAN/blob/master/SUPAHOTGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**INSTALL MAGENTA**

In [3]:
#@test {"output": "ignore"}

print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi

# Temporary hack since the colab installs a RC version of tensorflow.
!pip uninstall -y tensorflow
!pip install -q magenta-gpu

# Hack to allow python to pick up the newly-installed fluidsynth lib. 
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

print('Importing libraries and defining some helper functions...')
from google.colab import files

import magenta.music as mm
import magenta
import tensorflow

print('🎉 Done!')
print(magenta.__version__ )
print(tensorflow.__version__)

Installing dependencies...
Uninstalling tensorflow-1.12.0:
  Successfully uninstalled tensorflow-1.12.0
tcmalloc: large alloc 1073750016 bytes == 0x4faea000 @  0x7ffb713b22a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
Importing libraries and defining some helper functions...
🎉 Done!
0.3.19
1.12.0


**GAN**

In [0]:
from google.colab import files

In [0]:
uploaded = files.upload()
# Upload dataset. Download from Readme.md if needed.
# TAKES A LONG TIME

Saving 0a0ce238fb8c672549f77f3b692ebf32.mid to 0a0ce238fb8c672549f77f3b692ebf32 (2).mid
Saving 0a0dadc29a6ad3bec746c8b5af9c41cd.mid to 0a0dadc29a6ad3bec746c8b5af9c41cd (2).mid
Saving 0a0fff95b40e93a4b68a368ffb3f3dc7.mid to 0a0fff95b40e93a4b68a368ffb3f3dc7 (2).mid
Saving 0a1a1fe76071df5be3ddf65c76397e2d.mid to 0a1a1fe76071df5be3ddf65c76397e2d (2).mid
Saving 0a1afe828e4fb1ea6238366b02410b7a.mid to 0a1afe828e4fb1ea6238366b02410b7a (1).mid
Saving 0a1b7f59058eb2fba0a5bf43295c638d.mid to 0a1b7f59058eb2fba0a5bf43295c638d (1).mid
Saving 0a1de222f0bb6230a78e695ebd4fd3cc.mid to 0a1de222f0bb6230a78e695ebd4fd3cc (1).mid
Saving 0a1eb612307a7e52db3f31f34382b9c6.mid to 0a1eb612307a7e52db3f31f34382b9c6 (1).mid
Saving 0a2ad296a19a7e25fd67b535e8b51f7e.mid to 0a2ad296a19a7e25fd67b535e8b51f7e (1).mid
Saving 0a2b5be9a7cb2b743d01e9eb72078c84.mid to 0a2b5be9a7cb2b743d01e9eb72078c84 (1).mid
Saving 0a2ecc8127c1818903bd794a690129f2.mid to 0a2ecc8127c1818903bd794a690129f2 (1).mid
Saving 0a3b4de9a197ba9617424bfde

In [10]:
print(len(uploaded))

4


Sample GAN from: https://skymind.ai/wiki/generative-adversarial-network-gan for use on MNIST images to show a working GAN.

In [0]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

In [0]:
class Gan():
  def __init__(self):
    self.img_rows = 28
    self.img_cols = 28
    self.channels = 1
    self.img_shape = (self.img_rows, self.img_cols, self.channels)
    optimizer = Adam(0.0002, 0.5)
    
    # Discriminator
    self.discriminator = self.build_discriminator()
    self.discriminator.compile(
        loss='binary_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy'])
    
    # Generator
    self.generator = self.build_generator()
    self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)
    
    z = Input(shape=(100,))
    img = self.generator(z)
    
    self.discriminator.trainable = False
    
    valid = self.discriminator(img)
    
    self.combined = Model(z, valid)
    self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
    
  def build_generator(self):
    noise_shape = (100,)
    
    model = Sequential()
    
    model.add(Dense(256, input_shape = noise_shape))
    model.add(LeakyReLu(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(self.img_shape), activation='tanh'))
    model.add(Reshape(self.img_shape))
    
    model.summary()
    
    noise = Input(shape=noise_shape)
    img = model(noise)
    
    return Model(noise, img)
    
  def build_discriminator(self):
    img_shape = (self.img_rows, self.img_cols, self.channels)
    model = Sequential()
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLu(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLu(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    
    img = Input(shape=img_shape)
    validity = model(img)
    
    return Model(img, validity)
  
  def train(self, epochs, batch_size=128, save_interval=50):
    (X_train, _), (_,_) = mnist.load_data()
    
    X_train = (X_train.astype(np.float32) - 127.5)/127.5
    X_train = np.expand_dims(X_train, axis=3)
    
    half_batch = int(batch_size / 2)
    
    for epoch in range(epochs):
      # Discriminator
      idx = np.random.randint(0, X_train.shape[0], half_batch)
      imgs = X_train[idx]
      
      noise = np.random.normal(0, 1, (half_batch, 100))
      
      gen_imgs = self.generator.predict(noise)
      
      d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
      d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
      
      # Generator
      noise = np.random.normal(0, 1, (batch_size, 100))
      valid_y = np.array([1] * batch_size)
      
      g_loss = self.combined.train_on_batch(noise, valid_y)
      
      # plot progress
      print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
      
      # If at save interval => save generated image samples
      if epoch % save_interval == 0:
         self.save_imgs(epoch)
          
    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("gan/images/mnist_%d.png" % epoch)
        files.download("gan/images/mnist_%d.png" % epoch)
        plt.close()
    